In [2]:
# Librerias esenciales
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings

# For interactive visualizations
import plotly.io as pio

pio.templates.default = "plotly_white"

# Suppress warnings
warnings.filterwarnings("ignore")

# Set styling
sns.set(style="whitegrid")
plt.style.use("fivethirtyeight")

## Carga de datos y exploracion inicial

In [3]:
# Cargar los datos
df = pd.read_csv(
    "../data/raw/Vert_CITES_2000_2021.csv", encoding="latin-1", low_memory=False
)

# Vistazo general de los datos
print(f"Dimensiones del DataFrame: {df.shape}")
print(f"\nUso de memoria: {df.memory_usage(deep=True).sum() / (1024**2):.2f} MB")
print("\nPrimeras filas:")
df.head()

Dimensiones del DataFrame: (2812313, 19)

Uso de memoria: 2594.47 MB

Primeras filas:


,Unnamed: 0,Taxon,Class,IUCN_code,IUCNName,Name_for_CITESdb,Year,Threat_code,Source_clean,Live,n,Exporter,Exporter_Country,Exporter_region,Exporter_subregion,Importer,Importer_Country,Importer_region,Importer_subregion
0,2,Abronia aurita,Reptilia,EN,Abronia aurita,Abronia aurita,18,Threatened,Wild,Not live,0.0,US,United States of America,Americas,Northern America,CA,Canada,Americas,Northern America
1,3,Abronia aurita,Reptilia,EN,Abronia aurita,Abronia aurita,19,Threatened,Wild,Not live,0.0,US,United States of America,Americas,Northern America,CA,Canada,Americas,Northern America
2,4,Abronia aurita,Reptilia,EN,Abronia aurita,Abronia aurita,17,Threatened,Wild,Not live,0.0,US,United States of America,Americas,Northern America,CA,Canada,Americas,Northern America
3,5,Abronia aurita,Reptilia,EN,Abronia aurita,Abronia aurita,18,Threatened,Captive,Live,1.0,US,United States of America,Americas,Northern America,CA,Canada,Americas,Northern America
4,6,Abronia aurita,Reptilia,EN,Abronia aurita,Abronia aurita,21,Threatened,Captive,Live,0.0,US,United States of America,Americas,Northern America,CA,Canada,Americas,Northern America


### Informacion del dataset

In [4]:
# Mostrar información del dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2812313 entries, 0 to 2812312
Data columns (total 19 columns):
 #   Column              Dtype  
---  ------              -----  
 0   Unnamed: 0          int64  
 1   Taxon               object 
 2   Class               object 
 3   IUCN_code           object 
 4   IUCNName            object 
 5   Name_for_CITESdb    object 
 6   Year                int64  
 7   Threat_code         object 
 8   Source_clean        object 
 9   Live                object 
 10  n                   float64
 11  Exporter            object 
 12  Exporter_Country    object 
 13  Exporter_region     object 
 14  Exporter_subregion  object 
 15  Importer            object 
 16  Importer_Country    object 
 17  Importer_region     object 
 18  Importer_subregion  object 
dtypes: float64(1), int64(2), object(16)
memory usage: 407.7+ MB


### Resumen estadistico

In [5]:
# Resumen estadistico por el numero de columnas
df.describe().T.style.background_gradient(cmap="Blues")

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,2812313.000000,3339780.160763,3102651.235674,1.000000,703079.000000,2284401.000000,5343913.000000,11221711.000000
Year,2812313.000000,10.492690,6.347785,0.000000,5.000000,10.000000,16.000000,21.000000
n,2812313.000000,35.425873,2037.816502,0.000000,0.000000,0.000000,0.000000,1420223.000000


Se observa que hay valores entre 0 y 1420223 en la columna 'n', posibles outliers

### Revisar valores que faltan

In [6]:
# Checkeo de valores nulos
valores_nulos = df.isnull().sum()
porcentaje_nulos = (valores_nulos / len(df)) * 100

nulos_df = pd.DataFrame(
    {"Valores Nulos": valores_nulos, "Porcentaje Nulos": porcentaje_nulos}
)

print("\nValores nulos por columna:")
nulos_df[nulos_df["Valores Nulos"] > 0]


Valores nulos por columna:


,Valores Nulos,Porcentaje Nulos
IUCNName,1716,0.061017


In [7]:
# Borrar animales sin IUCNName
df = df.dropna(subset=["IUCNName"])

### Identificar especies

In [8]:
# Identificar las clases de especies
clases_especies = df["Class"].unique()
clases_especies

array(['Reptilia', 'Aves', 'Mammalia', 'Amphibia'], dtype=object)

In [9]:
# Renombrar las clases de especies para mayor claridad
clases_especies_renombradas = {
    "Amphibia": "Anfibios",
    "Aves": "Aves",
    "Mammalia": "Mamíferos",
    "Reptilia": "Reptiles",
}
df["Class"] = df["Class"].replace(clases_especies_renombradas)
df.head()

,Unnamed: 0,Taxon,Class,IUCN_code,IUCNName,Name_for_CITESdb,Year,Threat_code,Source_clean,Live,n,Exporter,Exporter_Country,Exporter_region,Exporter_subregion,Importer,Importer_Country,Importer_region,Importer_subregion
0,2,Abronia aurita,Reptiles,EN,Abronia aurita,Abronia aurita,18,Threatened,Wild,Not live,0.0,US,United States of America,Americas,Northern America,CA,Canada,Americas,Northern America
1,3,Abronia aurita,Reptiles,EN,Abronia aurita,Abronia aurita,19,Threatened,Wild,Not live,0.0,US,United States of America,Americas,Northern America,CA,Canada,Americas,Northern America
2,4,Abronia aurita,Reptiles,EN,Abronia aurita,Abronia aurita,17,Threatened,Wild,Not live,0.0,US,United States of America,Americas,Northern America,CA,Canada,Americas,Northern America
3,5,Abronia aurita,Reptiles,EN,Abronia aurita,Abronia aurita,18,Threatened,Captive,Live,1.0,US,United States of America,Americas,Northern America,CA,Canada,Americas,Northern America
4,6,Abronia aurita,Reptiles,EN,Abronia aurita,Abronia aurita,21,Threatened,Captive,Live,0.0,US,United States of America,Americas,Northern America,CA,Canada,Americas,Northern America


### Exploracion Threat_code

In [10]:
# Identificar threat_code
threat_codes = df["Threat_code"].unique()
threat_codes

array(['Threatened', 'Non-threatened', 'Not assessed', 'Extinct'],
      dtype=object)

In [11]:
# Renombrar los threat_codes para mayor claridad
threat_codes_renombrados = {
    "Threatened": "Amenazada",
    "Non-threatened": "No amenazada",
    "Not assessed": "No evaluada",
    "Extinct": "Extinta",
}
df["Threat_code"] = df["Threat_code"].replace(threat_codes_renombrados)

df.head()

,Unnamed: 0,Taxon,Class,IUCN_code,IUCNName,Name_for_CITESdb,Year,Threat_code,Source_clean,Live,n,Exporter,Exporter_Country,Exporter_region,Exporter_subregion,Importer,Importer_Country,Importer_region,Importer_subregion
0,2,Abronia aurita,Reptiles,EN,Abronia aurita,Abronia aurita,18,Amenazada,Wild,Not live,0.0,US,United States of America,Americas,Northern America,CA,Canada,Americas,Northern America
1,3,Abronia aurita,Reptiles,EN,Abronia aurita,Abronia aurita,19,Amenazada,Wild,Not live,0.0,US,United States of America,Americas,Northern America,CA,Canada,Americas,Northern America
2,4,Abronia aurita,Reptiles,EN,Abronia aurita,Abronia aurita,17,Amenazada,Wild,Not live,0.0,US,United States of America,Americas,Northern America,CA,Canada,Americas,Northern America
3,5,Abronia aurita,Reptiles,EN,Abronia aurita,Abronia aurita,18,Amenazada,Captive,Live,1.0,US,United States of America,Americas,Northern America,CA,Canada,Americas,Northern America
4,6,Abronia aurita,Reptiles,EN,Abronia aurita,Abronia aurita,21,Amenazada,Captive,Live,0.0,US,United States of America,Americas,Northern America,CA,Canada,Americas,Northern America


In [12]:
# Conteo por threat_code
conteo_threat_code = df["Threat_code"].value_counts()
conteo_threat_code

Threat_code
No amenazada    1459767
No evaluada      817561
Amenazada        532807
Extinta             462
Name: count, dtype: int64

In [13]:
# Conteo por especie y threat_copde (ordenado de mayor a menor y con índice reiniciado)
threat_especies = (
    df.groupby(["IUCNName", "Threat_code"])
    .size()
    .reset_index(name="conteo")
    .sort_values("conteo", ascending=False)
    .reset_index(drop=True)
)
threat_especies.head(10)

,IUCNName,Threat_code,conteo
0,Crocodylus niloticus,No evaluada,31312
1,Boa constrictor,No evaluada,28140
2,Panthera leo,Amenazada,28072
3,Ara ararauna,No amenazada,25960
4,Loxodonta africana,No evaluada,25872
5,Centrochelys sulcata,Amenazada,22176
6,Canis lupus,No amenazada,19888
7,Amazona aestiva,No amenazada,19800
8,Chelonoidis carbonarius,No evaluada,19712
9,Leptailurus serval,No amenazada,18128


### Exploracion codigo IUCN

In [14]:
# Identificar el codigo IUCN
codigos_iucn = df["IUCN_code"].unique()
codigos_iucn

array(['EN', 'LC', 'VU', 'Not assessed', 'CR', 'NT', 'EX', 'DD', 'EW'],
      dtype=object)

In [15]:
# Renombrar los codigos IUCN para mayor claridad
codigos_iucn_renombrados = {
    "LC": "Preocupación menor",
    "NT": "Casi amenazada",
    "VU": "Vulnerable",
    "EN": "En Peligro",
    "CR": "En Peligro Crítico",
    "EW": "Extinto en estado silvestre",
    "EX": "Extinto",
    "DD": "Datos insuficientes",
    "Not assessed": "Datos insuficientes",
}
df["IUCN_code"] = df["IUCN_code"].replace(codigos_iucn_renombrados)

df.head()

,Unnamed: 0,Taxon,Class,IUCN_code,IUCNName,Name_for_CITESdb,Year,Threat_code,Source_clean,Live,n,Exporter,Exporter_Country,Exporter_region,Exporter_subregion,Importer,Importer_Country,Importer_region,Importer_subregion
0,2,Abronia aurita,Reptiles,En Peligro,Abronia aurita,Abronia aurita,18,Amenazada,Wild,Not live,0.0,US,United States of America,Americas,Northern America,CA,Canada,Americas,Northern America
1,3,Abronia aurita,Reptiles,En Peligro,Abronia aurita,Abronia aurita,19,Amenazada,Wild,Not live,0.0,US,United States of America,Americas,Northern America,CA,Canada,Americas,Northern America
2,4,Abronia aurita,Reptiles,En Peligro,Abronia aurita,Abronia aurita,17,Amenazada,Wild,Not live,0.0,US,United States of America,Americas,Northern America,CA,Canada,Americas,Northern America
3,5,Abronia aurita,Reptiles,En Peligro,Abronia aurita,Abronia aurita,18,Amenazada,Captive,Live,1.0,US,United States of America,Americas,Northern America,CA,Canada,Americas,Northern America
4,6,Abronia aurita,Reptiles,En Peligro,Abronia aurita,Abronia aurita,21,Amenazada,Captive,Live,0.0,US,United States of America,Americas,Northern America,CA,Canada,Americas,Northern America


In [16]:
# Conteo por estado IUCN_code
conteo_iucn = df["IUCN_code"].value_counts()
conteo_iucn

IUCN_code
Preocupación menor             1264138
Datos insuficientes             817561
Vulnerable                      327877
Casi amenazada                  195629
En Peligro                      149733
En Peligro Crítico               52461
Extinto en estado silvestre       2736
Extinto                            462
Name: count, dtype: int64

In [17]:
# Conteo por especie y estado (ordenado de mayor a menor y con índice reiniciado)
frecuencia_especies = (
    df.groupby(["IUCNName", "IUCN_code"])
    .size()
    .reset_index(name="conteo")
    .sort_values("conteo", ascending=False)
    .reset_index(drop=True)
)
frecuencia_especies.head(10)

,IUCNName,IUCN_code,conteo
0,Crocodylus niloticus,Datos insuficientes,31312
1,Boa constrictor,Datos insuficientes,28140
2,Panthera leo,Vulnerable,28072
3,Ara ararauna,Preocupación menor,25960
4,Loxodonta africana,Datos insuficientes,25872
5,Centrochelys sulcata,Vulnerable,21168
6,Canis lupus,Preocupación menor,19888
7,Chelonoidis carbonarius,Datos insuficientes,19712
8,Leptailurus serval,Preocupación menor,18128
9,Python sebae,Datos insuficientes,17892


### HAGAN LOS ANALISIS DE FORMA ORDENADA Y CON MARKDOWNS PARA QUE SEA MAS FACIL DE LEER

#### ✅Borrar tabla Unnamed: 0
#### ✅Hacer los años mas legibles -> (18=2018, 19=2019...)
#### ✅Simplicar Exporter, Exporter_Country, Exporter_region, Importer, Importer_Country, Importer_region..., muestran la misma informacion
#### ✅Hacer conteo de los paises que exportan e importan de la misma manera que se hizo con las especies, IUCN y clases
#### ✅Hacer conteo del "taxon" "IUCNName" que es el nombre cientifico de la especie
#### ✅Hacer un conteo de las especies de las especies y su IUCN_code
#### ✅Existen 1716 registros en IUCN_code que no tienen un valor asignado, ¿Eliminar, reemplazar por no evaluado o verificar que este bien escrito?
#### ✅Hacer conteo de la columna "Live" y renombrar los valores para mayor claridad
#### ✅Hagan top de paises importadores y exportadores, y de las especies mas traficadas
#### ✅% de especies traficadas vivas vs muertas
#### 🚨Si tienen dudas pregunten por whatsapp o en el canal de discord, no se queden con dudas.

#### Recomendaciones para el análisis de la columna "n"

Con base en el análisis realizado, aquí están las recomendaciones para trabajar con esta columna:

#### 📊 **Interpretación de la columna "n":**
- **Significado**: Representa la **cantidad de especímenes** o **número de unidades** comercializadas en cada transacción
- **Valores cero (95.5%)**: Registros donde no se especificó la cantidad o donde la información no estaba disponible
- **Valores positivos**: Cantidades específicas de animales, productos derivados, o material biológico

#### 🔍 **Análisis sugeridos:**
1. **Separar análisis**: Crear análisis separados para registros con `n > 0` vs `n = 0`
2. **Volumen de comercio**: Usar la suma de `n` para calcular el volumen total comercializado
3. **Tipos de transacciones**: Los valores altos podrían indicar comercio de productos derivados (pieles, huevos)
4. **Intensidad por especie**: Calcular el promedio de `n` por especie para identificar patrones de comercio

#### ⚠️ **Consideraciones importantes:**
- Los valores cero no significan "cero comercio", sino "cantidad no especificada"
- Los valores extremos (>100,000) deben investigarse por posibles errores de datos
- Esta columna es clave para entender la **escala e intensidad** del tráfico de fauna

#### ❓ ¿Es recomendable retirar la columna "n" del análisis?

Evaluemos los pros y contras de mantener o retirar esta columna del análisis.

#### 🎯 **RECOMENDACIÓN FINAL: NO retirar la columna "n"**

Aunque el 95.5% de los registros tienen `n = 0`, **es altamente recomendable MANTENER esta columna** por las siguientes razones:

#### ✅ **Argumentos para MANTENERLA:**

1. **🔢 Datos valiosos disponibles**: 
   - 126,887 registros (4.5%) contienen información específica de cantidades
   - Representa información sobre volúmenes reales de comercio

2. **📊 Permite análisis diferenciados**:
   - Análisis de **intensidad** vs **frecuencia** del comercio
   - Identificación de operaciones de gran escala vs comercio individual
   - Cálculo de volúmenes totales por especie/país/año

3. **🎯 Información crucial para políticas públicas**:
   - Diferencia entre "1 tigre" y "10,000 pieles de reptil"
   - Identificación de patrones de comercio masivo
   - Priorización de esfuerzos de conservación

4. **🔍 Detección de patrones importantes**:
   - Especies con comercio de alto volumen
   - Países con operaciones comerciales masivas
   - Tendencias temporales en volúmenes

#### ⚠️ **Estrategia recomendada:**

**Enfoque de ANÁLISIS DUAL:**
- **Análisis 1**: Todos los registros (frecuencia de comercio)
- **Análisis 2**: Solo registros con `n > 0` (volúmenes de comercio)

Esto te permitirá obtener una visión completa: tanto la frecuencia como la intensidad del tráfico de fauna.

#### 💡 **Cómo usar la columna "n" eficientemente:**
1. Mantener todos los registros para análisis de frecuencia
2. Crear subconjuntos con `n > 0` para análisis de volumen
3. Usar visualizaciones separadas para cada tipo de análisis
4. Incluir ambas perspectivas en las conclusiones finales

In [18]:
# EJEMPLO PRÁCTICO: Implementación del análisis dual recomendado
print("=== EJEMPLO DE ANÁLISIS DUAL CON LA COLUMNA 'n' ===")

# Crear los dos datasets para análisis diferenciados
df_todos = df.copy()  # Para análisis de frecuencia
df_volumenes = df[df["n"] > 0].copy()  # Para análisis de volúmenes

print("📊 DATASET COMPLETO (análisis de frecuencia):")
print("• Registros totales: {len(df_todos):,}")
print("• Para analizar: frecuencia de comercio, rutas, especies más comercializadas")

print("\n📈 DATASET CON VOLÚMENES (análisis de intensidad):")
print(f"• Registros con cantidad: {len(df_volumenes):,}")
print("• Para analizar: volúmenes totales, operaciones masivas, intensidad por especie")

# Ejemplo de análisis comparativo
print("\n🔍 EJEMPLO COMPARATIVO - TOP 5 ESPECIES:")
print("\nPor FRECUENCIA (número de transacciones):")
top_frecuencia = df_todos["Taxon"].value_counts().head()
print(top_frecuencia)

print("\nPor VOLUMEN (especímenes comercializados):")
top_volumen = (
    df_volumenes.groupby("Taxon")["n"].sum().sort_values(ascending=False).head()
)
print(top_volumen)

print("\n💡 INSIGHT: ¡Diferentes especies dominan en frecuencia vs volumen!")
print("Esto demuestra la importancia de mantener ambos análisis.")

=== EJEMPLO DE ANÁLISIS DUAL CON LA COLUMNA 'n' ===
📊 DATASET COMPLETO (análisis de frecuencia):
• Registros totales: {len(df_todos):,}
• Para analizar: frecuencia de comercio, rutas, especies más comercializadas

📈 DATASET CON VOLÚMENES (análisis de intensidad):
• Registros con cantidad: 126,830
• Para analizar: volúmenes totales, operaciones masivas, intensidad por especie

🔍 EJEMPLO COMPARATIVO - TOP 5 ESPECIES:

Por FRECUENCIA (número de transacciones):
Taxon
Crocodylus niloticus    36256
Python regius           30976
Boa constrictor         29480
Panthera_leo            28072
Psittacus_erithacus     27302
Name: count, dtype: int64

Por VOLUMEN (especímenes comercializados):
Taxon
Varanus salvator              9865057.82
Iguana iguana                 8841484.00
Alligator mississippiensis    7580342.00
Malayopython reticulatus      6969494.00
Gekko gecko                   4170941.00
Name: n, dtype: float64

💡 INSIGHT: ¡Diferentes especies dominan en frecuencia vs volumen!
Esto demue

### Exploracion Year

In [19]:
# Uso de loc para Corregir y convertir años
# Solo transformar si el valor es menor a 100
df.loc[df["Year"] < 100, "Year"] = df["Year"] + 2000
df["Year"] = df["Year"].astype(int)
df

,Unnamed: 0,Taxon,Class,IUCN_code,IUCNName,Name_for_CITESdb,Year,Threat_code,Source_clean,Live,n,Exporter,Exporter_Country,Exporter_region,Exporter_subregion,Importer,Importer_Country,Importer_region,Importer_subregion
0,2,Abronia aurita,Reptiles,En Peligro,Abronia aurita,Abronia aurita,2018,Amenazada,Wild,Not live,0.0,US,United States of America,Americas,Northern America,CA,Canada,Americas,Northern America
1,3,Abronia aurita,Reptiles,En Peligro,Abronia aurita,Abronia aurita,2019,Amenazada,Wild,Not live,0.0,US,United States of America,Americas,Northern America,CA,Canada,Americas,Northern America
2,4,Abronia aurita,Reptiles,En Peligro,Abronia aurita,Abronia aurita,2017,Amenazada,Wild,Not live,0.0,US,United States of America,Americas,Northern America,CA,Canada,Americas,Northern America
3,5,Abronia aurita,Reptiles,En Peligro,Abronia aurita,Abronia aurita,2018,Amenazada,Captive,Live,1.0,US,United States of America,Americas,Northern America,CA,Canada,Americas,Northern America
4,6,Abronia aurita,Reptiles,En Peligro,Abronia aurita,Abronia aurita,2021,Amenazada,Captive,Live,0.0,US,United States of America,Americas,Northern America,CA,Canada,Americas,Northern America
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2812308,1280523,Tylototriton shanjing,Anfibios,Vulnerable,Tylototriton shanjing,Tylototriton shanjing,2020,Amenazada,Captive,Not live,0.0,CZ,Czechia,Europe,Eastern Europe,JP,Japan,Asia,Eastern Asia
2812309,1280533,Tylototriton shanjing,Anfibios,Vulnerable,Tylototriton shanjing,Tylototriton shanjing,2021,Amenazada,Captive,Not live,0.0,CZ,Czechia,Europe,Eastern Europe,JP,Japan,Asia,Eastern Asia
2812310,1280543,Tylototriton shanjing,Anfibios,Casi amenazada,Tylototriton shanjing,Tylototriton shanjing,2019,No amenazada,Captive,Not live,0.0,CZ,Czechia,Europe,Eastern Europe,KR,"Korea, Republic of",Asia,Eastern Asia
2812311,1280553,Tylototriton shanjing,Anfibios,Vulnerable,Tylototriton shanjing,Tylototriton shanjing,2020,Amenazada,Captive,Not live,0.0,CZ,Czechia,Europe,Eastern Europe,KR,"Korea, Republic of",Asia,Eastern Asia


### Exploracion Live

In [20]:
# Normalizamos algunos valores y renombramos
df["Live"] = df["Live"].replace({"Live": "Vivo", "Not live": "No vivo"})
df

,Unnamed: 0,Taxon,Class,IUCN_code,IUCNName,Name_for_CITESdb,Year,Threat_code,Source_clean,Live,n,Exporter,Exporter_Country,Exporter_region,Exporter_subregion,Importer,Importer_Country,Importer_region,Importer_subregion
0,2,Abronia aurita,Reptiles,En Peligro,Abronia aurita,Abronia aurita,2018,Amenazada,Wild,No vivo,0.0,US,United States of America,Americas,Northern America,CA,Canada,Americas,Northern America
1,3,Abronia aurita,Reptiles,En Peligro,Abronia aurita,Abronia aurita,2019,Amenazada,Wild,No vivo,0.0,US,United States of America,Americas,Northern America,CA,Canada,Americas,Northern America
2,4,Abronia aurita,Reptiles,En Peligro,Abronia aurita,Abronia aurita,2017,Amenazada,Wild,No vivo,0.0,US,United States of America,Americas,Northern America,CA,Canada,Americas,Northern America
3,5,Abronia aurita,Reptiles,En Peligro,Abronia aurita,Abronia aurita,2018,Amenazada,Captive,Vivo,1.0,US,United States of America,Americas,Northern America,CA,Canada,Americas,Northern America
4,6,Abronia aurita,Reptiles,En Peligro,Abronia aurita,Abronia aurita,2021,Amenazada,Captive,Vivo,0.0,US,United States of America,Americas,Northern America,CA,Canada,Americas,Northern America
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2812308,1280523,Tylototriton shanjing,Anfibios,Vulnerable,Tylototriton shanjing,Tylototriton shanjing,2020,Amenazada,Captive,No vivo,0.0,CZ,Czechia,Europe,Eastern Europe,JP,Japan,Asia,Eastern Asia
2812309,1280533,Tylototriton shanjing,Anfibios,Vulnerable,Tylototriton shanjing,Tylototriton shanjing,2021,Amenazada,Captive,No vivo,0.0,CZ,Czechia,Europe,Eastern Europe,JP,Japan,Asia,Eastern Asia
2812310,1280543,Tylototriton shanjing,Anfibios,Casi amenazada,Tylototriton shanjing,Tylototriton shanjing,2019,No amenazada,Captive,No vivo,0.0,CZ,Czechia,Europe,Eastern Europe,KR,"Korea, Republic of",Asia,Eastern Asia
2812311,1280553,Tylototriton shanjing,Anfibios,Vulnerable,Tylototriton shanjing,Tylototriton shanjing,2020,Amenazada,Captive,No vivo,0.0,CZ,Czechia,Europe,Eastern Europe,KR,"Korea, Republic of",Asia,Eastern Asia


In [21]:
# Conteo y porcentaje de animales vivos vs muertos
conteo_live = df["Live"].value_counts().rename_axis("Estado").reset_index(name="Conteo")
conteo_live["Porcentaje"] = conteo_live["Conteo"] / conteo_live["Conteo"].sum() * 100
conteo_live


,Estado,Conteo,Porcentaje
0,Vivo,1924669,68.47901
1,No vivo,885928,31.52099


### Exploracion paises exportadores e importadores

In [22]:
df["Importer_Country"].unique()

array(['Canada', 'Czechia', 'France', 'Korea, Republic of', 'Japan',
       'Switzerland', 'Indonesia', 'Thailand', 'Germany',
       'Iran (Islamic Republic of)',
       'United Kingdom of Great Britain and Northern Ireland', 'Sweden',
       'United States of America', 'Finland', 'India', 'Italy',
       'Russian Federation', 'Qatar', 'China', 'Réunion',
       'Taiwan, Province of China', 'Hong Kong', 'Mauritius', 'Malaysia',
       'Mozambique', 'Netherlands', 'Denmark', 'Spain', 'Belgium',
       'Bulgaria', 'United Arab Emirates', 'Austria', 'Australia',
       'Bahamas', 'Costa Rica', 'Israel', 'Virgin Islands (British)',
       'Viet Nam', 'Cyprus', 'Seychelles', 'Singapore',
       'Saint Helena, Ascension and Tristan da Cunha', 'South Africa',
       'Kazakhstan', 'Turkey', 'Paraguay', 'Saudi Arabia', 'Lebanon',
       'Mexico', 'Kuwait', 'Norway', 'Panama', 'Nigeria', 'Nicaragua',
       'Argentina', 'Aruba', 'Bangladesh', 'Bahrain',
       'Bolivia (Plurinational State of)'

In [23]:
# Crear funcion para estandarizar nombres de países
import re


def estandarizar_nombre(nombre):
    # Elimina paréntesis, comas, guiones, apóstrofes, puntos, corchetes y cualquier carácter especial salvo letras y espacios
    nombre = re.sub(r"[\,\(\)\[\]\'\-\.]", "", nombre)
    # Sustituye varios espacios por uno solo y elimina espacios iniciales/finales
    nombre = re.sub(r"\s+", " ", nombre).strip()
    return nombre


# Crea una columna auxiliar para nombres limpios de importadores y exportadores
df["Pais importador"] = df["Importer_Country"].apply(estandarizar_nombre)
df["Pais exportador"] = df["Exporter_Country"].apply(estandarizar_nombre)


In [24]:
# Hacemos un map para reemplazar los nombres de países
paises_limpio = {
    "United States of America": "Estados Unidos",
    "Mexico": "Mexico",
    "Czechia": "Chequia",
    "Germany": "Alemania",
    "South Africa": "Sudafrica",
    "Switzerland": "Suiza",
    "Netherlands": "Paises Bajos",
    "Cameroon": "Camerun",
    "Denmark": "Dinamarca",
    "Sweden": "Suecia",
    "Austria": "Austria",
    "Canada": "Canada",
    "Seychelles": "Seychelles",
    "Bulgaria": "Bulgaria",
    "Jordan": "Jordania",
    "Japan": "Japon",
    "Tanzania United Republic of": "Tanzania",
    "Mauritius": "Mauricio",
    "Malaysia": "Malasia",
    "Philippines": "Filipinas",
    "Syrian Arab Republic": "Siria",
    "Belarus": "Bielorrusia",
    "Argentina": "Argentina",
    "Israel": "Israel",
    "Korea Republic of": "Corea del Sur",
    "Papua New Guinea": "Papua Nueva Guinea",
    "China": "China",
    "Indonesia": "Indonesia",
    "Thailand": "Tailandia",
    "Viet Nam": "Vietnam",
    "United Kingdom of Great Britain and Northern Ireland": "Reino Unido",
    "Macao": "Macao",
    "Norway": "Noruega",
    "France": "Francia",
    "Ukraine": "Ucrania",
    "Slovenia": "Eslovenia",
    "Spain": "España",
    "Italy": "Italia",
    "Colombia": "Colombia",
    "Costa Rica": "Costa Rica",
    "Finland": "Finlandia",
    "Guatemala": "Guatemala",
    "Guyana": "Guyana",
    "Honduras": "Honduras",
    "Hungary": "Hungria",
    "Ireland": "Irlanda",
    "Jersey": "Jersey",
    "Luxembourg": "Luxemburgo",
    "Nicaragua": "Nicaragua",
    "Panama": "Panama",
    "Peru": "Peru",
    "Poland": "Polonia",
    "Russian Federation": "Rusia",
    "Slovakia": "Eslovaquia",
    "Suriname": "Surinam",
    "El Salvador": "El Salvador",
    "Trinidad and Tobago": "Trinidad y Tobago",
    "Uruguay": "Uruguay",
    "Uzbekistan": "Uzbekistan",
    "Venezuela Bolivarian Republic of": "Venezuela",
    "Armenia": "Armenia",
    "Aruba": "Aruba",
    "Belgium": "Belgica",
    "Brazil": "Brasil",
    "Mali": "Mali",
    "Fiji": "Fiyi",
    "Madagascar": "Madagascar",
    "Paraguay": "Paraguay",
    "Bolivia Plurinational State of": "Bolivia",
    "Benin": "Benin",
    "Congo Democratic Republic of the": "Republica Democratica del Congo",
    "Ghana": "Ghana",
    "Guinea": "Guinea",
    "Togo": "Togo",
    "Solomon Islands": "Islas Salomon",
    "Vanuatu": "Vanuatu",
    "United Arab Emirates": "Emiratos Arabes Unidos",
    "Burkina Faso": "Burkina Faso",
    "Egypt": "Egipto",
    "Bahrain": "Barein",
    "Sudan": "Sudan",
    "Senegal": "Senegal",
    "Niger": "Niger",
    "Reunion": "Reunion",
    "Libya": "Libia",
    "Uganda": "Uganda",
    "Ethiopia": "Etiopia",
    "Yemen": "Yemen",
    "Mozambique": "Mozambique",
    "Burundi": "Burundi",
    "Sierra Leone": "Sierra Leona",
    "Cote dIvoire": "Costa de Marfil",
    "Comoros": "Comoras",
    "Cayman Islands": "Islas Caiman",
    "Kenya": "Kenia",
    "Namibia": "Namibia",
    "Barbados": "Barbados",
    "Bonaire Sint Eustatius and Saba": "Bonaire San Eustaquio y Saba",
    "Curacao": "Curazao",
    "Antigua and Barbuda": "Antigua y Barbuda",
    "Hong Kong": "Hong Kong",
    "Saint Kitts and Nevis": "San Cristobal y Nieves",
    "Malta": "Malta",
    "Cuba": "Cuba",
    "India": "India",
    "Australia": "Australia",
    "Botswana": "Botsuana",
    "Kazakhstan": "Kazajistan",
    "Zambia": "Zambia",
    "Zimbabwe": "Zimbabue",
    "Malawi": "Malaui",
    "Tunisia": "Tunez",
    "Singapore": "Singapur",
    "Bangladesh": "Banglades",
    "Brunei Darussalam": "Brunei",
    "Cambodia": "Camboya",
    "Lao Peoples Democratic Republic": "Laos",
    "Sri Lanka": "Sri Lanka",
    "Bahamas": "Bahamas",
    "Myanmar": "Myanmar",
    "Portugal": "Portugal",
    "Greece": "Grecia",
    "Sint Maarten Dutch part": "Sint Maarten",
    "Gabon": "Gabon",
    "Equatorial Guinea": "Guinea Ecuatorial",
    "Turkey": "Turquia",
    "Ecuador": "Ecuador",
    "Croatia": "Croacia",
    "New Zealand": "Nueva Zelanda",
    "Latvia": "Letonia",
    "Montenegro": "Montenegro",
    "Estonia": "Estonia",
    "Guernsey": "Guernsey",
    "Chad": "Chad",
    "Saudi Arabia": "Arabia Saudita",
    "Azerbaijan": "Azerbaiyan",
    "Cyprus": "Chipre",
    "Algeria": "Argelia",
    "Iran Islamic Republic of": "Iran",
    "Lebanon": "Libano",
    "Morocco": "Marruecos",
    "Moldova Republic of": "Moldavia",
    "North Macedonia": "Macedonia del Norte",
    "Qatar": "Catar",
    "Monaco": "Monaco",
    "Romania": "Rumania",
    "Serbia": "Serbia",
    "San Marino": "San Marino",
    "Kuwait": "Kuwait",
    "Georgia": "Georgia",
    "Pakistan": "Pakistan",
    "Dominican Republic": "Republica Dominicana",
    "Sao Tome and Principe": "Santo Tome y Principe",
    "Bermuda": "Bermudas",
    "Chile": "Chile",
    "Belize": "Belice",
    "Oman": "Oman",
    "Jamaica": "Jamaica",
    "New Caledonia": "Nueva Caledonia",
    "Mongolia": "Mongolia",
    "Nigeria": "Nigeria",
    "Eswatini": "Esuatini",
    "Isle of Man": "Isla de Man",
    "Iraq": "Irak",
    "Iceland": "Islandia",
    "Dominica": "Dominica",
    "Greenland": "Groenlandia",
    "Kyrgyzstan": "Kirguistan",
    "Eritrea": "Eritrea",
    "Gambia": "Gambia",
    "Falkland Islands Malvinas": "Islas Malvinas",
    "Congo": "Republica del Congo",
    "Liberia": "Liberia",
    "GuineaBissau": "Guinea Bisau",
    "Central African Republic": "Republica Centroafricana",
    "Saint Lucia": "Santa Lucia",
    "Rwanda": "Ruanda",
    "Virgin Islands British": "Islas Virgenes Britanicas",
    "Lithuania": "Lituania",
    "Angola": "Angola",
}


In [25]:
# Crear columnas ya limpias
df["Pais importador"] = df["Pais importador"].replace(paises_limpio)
df["Pais exportador"] = df["Pais exportador"].replace(paises_limpio)

In [26]:
# Identificar valores de exportadores e importadores con porcentaje
exportadores = df["Pais exportador"].value_counts()
exportadores_porcentaje = (exportadores / exportadores.sum()) * 100
exportadores_df = pd.DataFrame(
    {"Conteo": exportadores, "Porcentaje": exportadores_porcentaje}
)

importadores = df["Pais importador"].value_counts()
importadores_porcentaje = (importadores / importadores.sum()) * 100
importadores_df = pd.DataFrame(
    {"Conteo": importadores, "Porcentaje": importadores_porcentaje}
)

In [27]:
exportadores_df.head(10)

,Conteo,Porcentaje
Pais exportador,,
Sudafrica,300908,10.706195
Estados Unidos,186082,6.620729
Indonesia,160990,5.727965
Paises Bajos,150960,5.371101
Canada,145710,5.184308
Alemania,115608,4.113290
Madagascar,93524,3.327549
Chequia,85610,3.045972
Belgica,75986,2.703554


In [28]:
importadores_df.head(10)

,Conteo,Porcentaje
Pais importador,,
Estados Unidos,169968,6.047398
Japon,165908,5.902945
Alemania,102672,3.653032
Canada,90022,3.202949
Taiwan Province of China,83234,2.961435
Reino Unido,79590,2.831783
Suiza,75506,2.686476
Corea del Sur,71888,2.557748
Tailandia,71736,2.552340


####

### Analisis regiones exportadoras e importadoras

In [29]:
df["Importer_region"].unique()

array(['Americas', 'Europe', 'Asia', 'Africa', 'Oceania'], dtype=object)

In [30]:
# Identificar valores de exportadores e importadores con porcentaje
exportadoresC = df["Exporter_region"].value_counts()
exportadoresC_porcentaje = (exportadoresC / exportadoresC.sum()) * 100
exportadoresC_df = pd.DataFrame(
    {"Conteo": exportadoresC, "Porcentaje": exportadoresC_porcentaje}
)

importadoresC = df["Importer_region"].value_counts()
importadoresC_porcentaje = (importadoresC / importadoresC.sum()) * 100
importadoresC_df = pd.DataFrame(
    {"Conteo": importadoresC, "Porcentaje": importadoresC_porcentaje}
)

In [31]:
importadoresC_df.head()

,Conteo,Porcentaje
Importer_region,,
Asia,1121834,39.914438
Europe,974636,34.677188
Americas,474449,16.880720
Africa,210266,7.481186
Oceania,29412,1.046468


In [32]:
exportadoresC_df.head()

,Conteo,Porcentaje
Exporter_region,,
Africa,914360,32.532590
Europe,775783,27.602072
Americas,666778,23.723714
Asia,417450,14.852716
Oceania,36226,1.288908


### Analisis subregiones importadoras y exportadoras

In [33]:
# Visualizar array de subregiones
df["Exporter_subregion"].unique(), df["Importer_subregion"].unique()

(array(['Northern America', 'Latin America and the Caribbean',
        'Eastern Europe', 'Western Europe', 'Sub-Saharan Africa',
        'Northern Europe', 'Western Asia', 'Eastern Asia',
        'South-eastern Asia', 'Melanesia', 'Southern Europe',
        'Central Asia', 'Northern Africa', 'Southern Asia',
        'Australia and New Zealand'], dtype=object),
 array(['Northern America', 'Eastern Europe', 'Western Europe',
        'Eastern Asia', 'South-eastern Asia', 'Southern Asia',
        'Northern Europe', 'Southern Europe', 'Western Asia',
        'Sub-Saharan Africa', 'Australia and New Zealand',
        'Latin America and the Caribbean', 'Central Asia',
        'Northern Africa', 'Polynesia', 'Melanesia', 'Micronesia'],
       dtype=object))

In [34]:
subregiones_esp = {
    "Northern America": "Norteamérica",
    "Latin America and the Caribbean": "Latinoamérica y Caribe",
    "Eastern Europe": "Europa Oriental",
    "Western Europe": "Europa Occidental",
    "Northern Europe": "Europa Septentrional",
    "Southern Europe": "Europa Meridional",
    "Central Asia": "Asia Central",
    "Eastern Asia": "Asia Oriental",
    "South-eastern Asia": "Sudeste Asiático",
    "Southern Asia": "Asia Meridional",
    "Western Asia": "Asia Occidental",
    "Northern Africa": "África del Norte",
    "Sub-Saharan Africa": "África Subsahariana",
    "Australia and New Zealand": "Australia y Nueva Zelanda",
    "Polynesia": "Polinesia",
    "Melanesia": "Melanesia",
    "Micronesia": "Micronesia",
}

# Realizar los reemplazos en las columnas de subregiones
df["Exporter_subregion"] = df["Exporter_subregion"].replace(subregiones_esp)
df["Importer_subregion"] = df["Importer_subregion"].replace(subregiones_esp)

In [35]:
# Identificar valores de exportadores e importadores con porcentaje
exportadoresS = df["Exporter_subregion"].value_counts()
exportadoresS_porcentaje = (exportadoresS / exportadoresS.sum()) * 100
exportadoresS_df = pd.DataFrame(
    {"Conteo": exportadoresS, "Porcentaje": exportadoresS_porcentaje}
)

importadoresS = df["Importer_subregion"].value_counts()
importadoresS_porcentaje = (importadoresS / importadoresS.sum()) * 100
importadoresS_df = pd.DataFrame(
    {"Conteo": importadoresS, "Porcentaje": importadoresS_porcentaje}
)

In [36]:
exportadoresS_df.head(10)

,Conteo,Porcentaje
Exporter_subregion,,
África Subsahariana,889518,31.648721
Europa Occidental,423599,15.071495
Norteamérica,339338,12.073520
Latinoamérica y Caribe,327440,11.650194
Sudeste Asiático,294936,10.493714
Europa Oriental,184762,6.573764
Europa Septentrional,102750,3.655807
Asia Occidental,71598,2.547430
Europa Meridional,64672,2.301006


In [37]:
importadoresS_df.head(10)

,Conteo,Porcentaje
Importer_subregion,,
Asia Oriental,454526,16.171867
Europa Occidental,382030,13.592486
Asia Occidental,307700,10.947852
Norteamérica,264246,9.401775
Sudeste Asiático,252255,8.975139
Latinoamérica y Caribe,210203,7.478945
Europa Oriental,203826,7.252054
Europa Septentrional,197574,7.029610
Europa Meridional,191206,6.803039


### Analisis Source_clean

### Borrar columnas innecesarias

In [ ]:
# Eliminar columnas redundantes
df = df.drop(
    columns=[
        "Exporter",
        "Importer",
        "Taxon",
        "Name_for_CITESdb",
        "Unnamed: 0",
        "Importer_Country",
        "Exporter_Country",
        "Source_clean",
    ]
)

df

,Class,IUCN_code,IUCNName,Year,Threat_code,Source_clean,Live,n,Exporter_region,Exporter_subregion,Importer_region,Importer_subregion,Pais importador,Pais exportador
0,Reptiles,En Peligro,Abronia aurita,2018,Amenazada,Salvaje,No vivo,0.0,Americas,Norteamérica,Americas,Norteamérica,Canada,Estados Unidos
1,Reptiles,En Peligro,Abronia aurita,2019,Amenazada,Salvaje,No vivo,0.0,Americas,Norteamérica,Americas,Norteamérica,Canada,Estados Unidos
2,Reptiles,En Peligro,Abronia aurita,2017,Amenazada,Salvaje,No vivo,0.0,Americas,Norteamérica,Americas,Norteamérica,Canada,Estados Unidos
3,Reptiles,En Peligro,Abronia aurita,2018,Amenazada,Cautiverio,Vivo,1.0,Americas,Norteamérica,Americas,Norteamérica,Canada,Estados Unidos
4,Reptiles,En Peligro,Abronia aurita,2021,Amenazada,Cautiverio,Vivo,0.0,Americas,Norteamérica,Americas,Norteamérica,Canada,Estados Unidos
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2812308,Anfibios,Vulnerable,Tylototriton shanjing,2020,Amenazada,Cautiverio,No vivo,0.0,Europe,Europa Oriental,Asia,Asia Oriental,Japon,Chequia
2812309,Anfibios,Vulnerable,Tylototriton shanjing,2021,Amenazada,Cautiverio,No vivo,0.0,Europe,Europa Oriental,Asia,Asia Oriental,Japon,Chequia
2812310,Anfibios,Casi amenazada,Tylototriton shanjing,2019,No amenazada,Cautiverio,No vivo,0.0,Europe,Europa Oriental,Asia,Asia Oriental,Corea del Sur,Chequia
2812311,Anfibios,Vulnerable,Tylototriton shanjing,2020,Amenazada,Cautiverio,No vivo,0.0,Europe,Europa Oriental,Asia,Asia Oriental,Corea del Sur,Chequia


### Renombrar columnas, convertir datos y exportar

In [ ]:
# Renombramos las columnas
# Nota: suceptible a cambios según consideraciones futuras
columnas_renombradas = {
    "Class": "Clase",
    "IUCN_code": "Categoria IUCN",
    "IUCNName": "Nombre cientifico",
    "Year": "Año",
    "Live": "Estado ejemplar",
    "n": "Cantidad",
    "Exporter_region": "Region exportadora",
    "Importer_region": "Region importadora",
    "Exporter_subregion": "Subregion exportadora",
    "Importer_subregion": "Subregion importadora",
    "Threat_code": "Estado amenaza",
}
df.rename(columns=columnas_renombradas, inplace=True)
df

,Clase,Categoria IUCN,Nombre cientifico,Año,Estado amenaza,Origen,Estado ejemplar,Cantidad,Region exportadora,Subregion exportadora,Region importadora,Subregion importadora,Pais importador,Pais exportador
0,Reptiles,En Peligro,Abronia aurita,2018,Amenazada,Salvaje,No vivo,0.0,Americas,Norteamérica,Americas,Norteamérica,Canada,Estados Unidos
1,Reptiles,En Peligro,Abronia aurita,2019,Amenazada,Salvaje,No vivo,0.0,Americas,Norteamérica,Americas,Norteamérica,Canada,Estados Unidos
2,Reptiles,En Peligro,Abronia aurita,2017,Amenazada,Salvaje,No vivo,0.0,Americas,Norteamérica,Americas,Norteamérica,Canada,Estados Unidos
3,Reptiles,En Peligro,Abronia aurita,2018,Amenazada,Cautiverio,Vivo,1.0,Americas,Norteamérica,Americas,Norteamérica,Canada,Estados Unidos
4,Reptiles,En Peligro,Abronia aurita,2021,Amenazada,Cautiverio,Vivo,0.0,Americas,Norteamérica,Americas,Norteamérica,Canada,Estados Unidos
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2812308,Anfibios,Vulnerable,Tylototriton shanjing,2020,Amenazada,Cautiverio,No vivo,0.0,Europe,Europa Oriental,Asia,Asia Oriental,Japon,Chequia
2812309,Anfibios,Vulnerable,Tylototriton shanjing,2021,Amenazada,Cautiverio,No vivo,0.0,Europe,Europa Oriental,Asia,Asia Oriental,Japon,Chequia
2812310,Anfibios,Casi amenazada,Tylototriton shanjing,2019,No amenazada,Cautiverio,No vivo,0.0,Europe,Europa Oriental,Asia,Asia Oriental,Corea del Sur,Chequia
2812311,Anfibios,Vulnerable,Tylototriton shanjing,2020,Amenazada,Cautiverio,No vivo,0.0,Europe,Europa Oriental,Asia,Asia Oriental,Corea del Sur,Chequia


In [ ]:
# Convertir columnas a un formato más adecuado
df["Clase"] = df["Clase"].astype("string")
df["Categoria IUCN"] = df["Categoria IUCN"].astype("string")
df["Nombre cientifico"] = df["Nombre cientifico"].astype("string")
df["Estado ejemplar"] = df["Estado ejemplar"].astype("string")
df["Region exportadora"] = df["Region exportadora"].astype("string")
df["Region importadora"] = df["Region importadora"].astype("string")
df["Subregion exportadora"] = df["Subregion exportadora"].astype("string")
df["Subregion importadora"] = df["Subregion importadora"].astype("string")
df["Pais exportador"] = df["Pais exportador"].astype("string")
df["Pais importador"] = df["Pais importador"].astype("string")
df["Estado amenaza"] = df["Estado amenaza"].astype("string")


In [ ]:
columnas_ordenadas = [
    "Nombre cientifico",
    "Clase",
    "Estado amenaza",
    "Categoria IUCN",
    "Estado ejemplar",
    "Cantidad",
    "Año",
    "Pais exportador",
    "Region exportadora",
    "Subregion exportadora",
    "Pais importador",
    "Region importadora",
    "Subregion importadora",
]
df = df[columnas_ordenadas]


In [45]:
df.head(10)

,Nombre cientifico,Clase,Estado amenaza,Categoria IUCN,Estado ejemplar,Cantidad,Año,Origen,Pais exportador,Region exportadora,Subregion exportadora,Pais importador,Region importadora,Subregion importadora
0,Abronia aurita,Reptiles,Amenazada,En Peligro,No vivo,0.0,2018,Salvaje,Estados Unidos,Americas,Norteamérica,Canada,Americas,Norteamérica
1,Abronia aurita,Reptiles,Amenazada,En Peligro,No vivo,0.0,2019,Salvaje,Estados Unidos,Americas,Norteamérica,Canada,Americas,Norteamérica
2,Abronia aurita,Reptiles,Amenazada,En Peligro,No vivo,0.0,2017,Salvaje,Estados Unidos,Americas,Norteamérica,Canada,Americas,Norteamérica
3,Abronia aurita,Reptiles,Amenazada,En Peligro,Vivo,1.0,2018,Cautiverio,Estados Unidos,Americas,Norteamérica,Canada,Americas,Norteamérica
4,Abronia aurita,Reptiles,Amenazada,En Peligro,Vivo,0.0,2021,Cautiverio,Estados Unidos,Americas,Norteamérica,Canada,Americas,Norteamérica
5,Abronia aurita,Reptiles,Amenazada,En Peligro,Vivo,0.0,2019,Cautiverio,Estados Unidos,Americas,Norteamérica,Canada,Americas,Norteamérica
6,Abronia aurita,Reptiles,Amenazada,En Peligro,Vivo,0.0,2020,Cautiverio,Estados Unidos,Americas,Norteamérica,Canada,Americas,Norteamérica
7,Abronia aurita,Reptiles,Amenazada,En Peligro,No vivo,0.0,2021,Salvaje,Estados Unidos,Americas,Norteamérica,Canada,Americas,Norteamérica
8,Abronia aurita,Reptiles,Amenazada,En Peligro,No vivo,0.0,2018,Cautiverio,Estados Unidos,Americas,Norteamérica,Canada,Americas,Norteamérica
9,Abronia aurita,Reptiles,Amenazada,En Peligro,No vivo,0.0,2017,Cautiverio,Estados Unidos,Americas,Norteamérica,Canada,Americas,Norteamérica


In [46]:
# Comprobar nuevo uso de memoria
print(f"\nUso de memoria: {df.memory_usage(deep=True).sum() / (1024**2):.2f} MB")


Uso de memoria: 2133.32 MB


In [47]:
df.to_csv(
    "../data/processed/trafico_fauna_limpio.csv", index=False, encoding="utf-8-sig"
)